<a href="https://colab.research.google.com/github/Somiya11/trial/blob/main/Copy_of_Finetuned_on_old_and_new_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 29.6 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from transformers import Trainer, TrainingArguments

In [ ]:
class SQLDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.questions = df['question'].tolist()
        self.queries = df['sql'].tolist()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        query = self.queries[idx]

        inputs = self.tokenizer.encode_plus(
            question,
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        labels = self.tokenizer.encode(
            query,
            padding='max_length',
            truncation=True,
            max_length=64,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': labels.squeeze()
        }

In [ ]:
df = pd.read_csv('cleaned_data_modified.csv')

In [ ]:
df=df.sample(frac=1)

In [ ]:
df.head()

,question,sql
54902,What is the average number of regencies (kabup...,SELECT AVG Number of Regencies (kabupaten) FRO...
23273,What is the Name when the League Cup shows 0 (...,SELECT Name FROM T1_name WHERE League Cup = 0 ...
45273,What is the number of tries for that has 30 tr...,SELECT Tries for FROM T1_table WHERE Tries aga...
44987,What was the date of Circuit Hockenheimring an...,SELECT Date FROM T1_player WHERE Circuit = hoc...
24388,What is average for Benito Lorenzi league when...,SELECT AVG League FROM T1_name WHERE Name = be...


In [ ]:
df=df.head(950)

In [ ]:
df2=pd.read_excel('NLtoSQL_dataset_part1.xlsx', sheet_name='Sheet1')

In [ ]:
df2.head()

,Natural_language,SQL
0,How many major codes are there in oltas?,SELECT DISTINCT(MAJ_HD_CD) FROM Table_oltas
1,How many minor codes are there in oltas?,SELECT DISTINCT(MINR_HD_CD) FROM Table_oltas
2,How many bank codes are there in oltas?,SELECT DISTINCT(BANK_CD) FROM Table_oltas
3,How many branch codes are there in oltas?,SELECT DISTINCT(BRANCH_CD) FROM Table_oltas
4,List taxpayers who have deposited challan in a...,SELECT DISTINCT(TAN_PAN) FROM Table_oltas WHER...


In [ ]:
df2.rename(columns={'Natural_language':'question', 'SQL':'sql'}, inplace=True)

In [ ]:
df_new=pd.concat([df, df2])

In [ ]:
df_new.shape

(1001, 2)

In [ ]:
train_data = df_new
valid_data = df_new[900:]
#test_data = df[45000:]

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
dataset1 = SQLDataset(train_data, tokenizer)
dataset2 = SQLDataset(valid_data, tokenizer)

In [ ]:
dataset1[0]

{'input_ids': tensor([  363,    19,     8,  1348,   381,    13,  5925, 11573,    41,  1258,
          3007,  4665,    35,    61,    28,    72,   145,   220,  3341, 12293,
            11,     3,     9,  2735,  2074, 23087,    13, 10440,  3916,  4482,
         26363,    58,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./output',
    num_train_epochs=5,
    per_device_train_batch_size=5,    #changed from 16 to 5
    per_device_eval_batch_size=5,
    learning_rate=1e-5,
    save_total_limit=1,
    fp16=True,  # Enable mixed-precision training if available
    logging_dir='./logs',
    logging_steps=10,  # changed from 500 to 10
    save_steps=500,
)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset1,
    eval_dataset=dataset2,
)


In [ ]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,11.639700
20,10.607600
30,9.044800
40,7.764200
50,6.671200
60,5.778300
70,4.993400
80,4.116300
90,3.347500
100,2.861900


TrainOutput(global_step=1005, training_loss=1.4120065976138139, metrics={'train_runtime': 232.2346, 'train_samples_per_second': 21.551, 'train_steps_per_second': 4.328, 'total_flos': 761958560563200.0, 'train_loss': 1.4120065976138139, 'epoch': 5.0})

In [ ]:
save_path = 'FinetunedT5_small_model'
!mkdir {save_path}
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('FinetunedT5_small_model/tokenizer_config.json',
 'FinetunedT5_small_model/special_tokens_map.json',
 'FinetunedT5_small_model/spiece.model',
 'FinetunedT5_small_model/added_tokens.json')

In [ ]:
!zip -r Finetuned_small.zip {save_path}

  adding: FinetunedT5_small_model/ (stored 0%)
  adding: FinetunedT5_small_model/pytorch_model.bin (deflated 9%)
  adding: FinetunedT5_small_model/spiece.model (deflated 48%)
  adding: FinetunedT5_small_model/special_tokens_map.json (deflated 86%)
  adding: FinetunedT5_small_model/tokenizer_config.json (deflated 83%)
  adding: FinetunedT5_small_model/config.json (deflated 62%)
  adding: FinetunedT5_small_model/generation_config.json (deflated 29%)


In [ ]:
output_dir = './fine-tuned-model'
trainer.save_model(output_dir)

In [ ]:
results = trainer.evaluate(dataset2)
print(results)

{'eval_loss': 1.1073660850524902, 'eval_runtime': 0.883, 'eval_samples_per_second': 114.387, 'eval_steps_per_second': 23.783, 'epoch': 5.0}


In [ ]:
input_question = "select average working hours of employee named aman"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad><extra_id_0>select average working hours of employee named aman. select average working hours of employee named aman. select average working hours of employee named aman. select average working hours of employee named aman. select average working hours of employee named aman. select average working hours of employee named


In [ ]:
input_question = "select average working hours of employee named aman"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Average working hours FROM T1_table WHERE Name = aman</s>


In [ ]:
input_question = "select average working hours of employee named aman"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())


In [ ]:
input_question = "what is the max salary of employee named aman"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT MAX Salary FROM T1_table WHERE Name = aman</s>


In [ ]:
input_question = "what is the PAN  of employee named aman"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT PAN FROM T1_table WHERE Name = aman</s>


In [ ]:
input_question = "Who is the winner of Cricket in 2022"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Winner FROM T1_player WHERE Year = 2022</s>


In [ ]:
input_question = "which the favourite song in India"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Favourite Song FROM T1_country WHERE Country = india</s>


In [ ]:
input_question = "which the favourite song in 2020"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Favorite song FROM T1_table WHERE Year = 2020</s>


In [ ]:
#generation from newly trained

In [ ]:
input_question = "what are the major codes in oltas"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Major Code FROM T1_table WHERE Major Code = oltas</s>


In [ ]:
input_question = "which the favourite song in India"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Favourite Song FROM T1_table WHERE Song = Hindi</s>


In [ ]:
input_question = "What is the average working hours of employee named Aman"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Working Hours FROM T1_table WHERE Employee = Amana</s>


In [ ]:
df_new

,question,sql
54902,What is the average number of regencies (kabup...,SELECT AVG Number of Regencies (kabupaten) FRO...
23273,What is the Name when the League Cup shows 0 (...,SELECT Name FROM T1_name WHERE League Cup = 0 ...
45273,What is the number of tries for that has 30 tr...,SELECT Tries for FROM T1_table WHERE Tries aga...
44987,What was the date of Circuit Hockenheimring an...,SELECT Date FROM T1_player WHERE Circuit = hoc...
24388,What is average for Benito Lorenzi league when...,SELECT AVG League FROM T1_name WHERE Name = be...
...,...,...
46,how much gross salary was filed by PANS ABCD12...,"SELECT B.PAN_NO, B.ASSESSMENT_YEAR, A.GROSS_IN..."
47,how much total tax paid was paid by PANS ABCD1...,"SELECT B.PAN_NO, B.ASSESSMENT_YEAR, A.TOTAL_TA..."
48,how much total tax paid was paid by PANS ABCD1...,"SELECT B.PAN_NO, B.ASSESSMENT_YEAR, A.TOTAL_TA..."
49,List all details filed in ITR1 by PANS ABCD123...,"SELECT B.PAN_NO, B.ASSESSMENT_YEAR, A.* from \..."


In [ ]:
input_question = "How much tax is paid by Aman"
input_encoded = tokenizer.encode_plus(
    input_question,
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
).to(device)

generated = model.generate(
    input_ids=input_encoded.input_ids,
    attention_mask=input_encoded.attention_mask,
    max_length=64,
    num_beams=4,
    early_stopping=True
)
generated_query = tokenizer.decode(generated.squeeze())

print("Generated SQL Query:", generated_query)

Generated SQL Query: <pad> SELECT Tax FROM T1_table WHERE Tax = Aman</s>
